# Chapter xx

*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

## Priority Queue

A priority queue is similar to a queue in the sense that the primary operations are adding and removing elements.
The difference is the "queue discipline", which determines which element you get when you remove one.
In queue, the discipline is FIFO, which means that when you remove an element, you get the one that was added first.

In a priority queue, the element you get is the one with the highest "priority", which is the one that comes first in the sort ordering of the elements.
For example, if the elements are strings, the element you get is the one that comes first in alphabetical order.
If the elements are numerical, you get the one with the lowest numerical value.

You can infer from this description that the element in the heap have to be ordered.
If they are not, you can get some strange behavior.

Possibly the simplest way to implement a priority queue is a list with the following two functions:

* `push:` Appends a new element to the end of the list.

* `pop`: Searches the list for the "smallest" element, removes it, and returns it.

In this implementation, we expect `push` to take constant time, but `pop` would take linear time.

We can get better performance, at least in the sense of asymptotic time, using a [heap](https://en.wikipedia.org/wiki/Heap_(data_structure)).
A heap is a binary tree that has the "heap property": if a node `P` is a parent of another node `C`, the value of `P` must come before the value of `C` in the sort order.
This property guarantees that the root of the tree contains the value that comes first in sort order.

This property suggests that can can find the element with highest priority in constant time, and that's true, but that doesn't mean we can implement `pop` in constant time, because when we remove the root node, we are left with two trees, and we have to do some work to join them into a single tree that has the heap property.

Similarly, when we add a new node to a heap, we have to do some work to add it in a way that restores the heap property.
In both cases, this additional work takes time proportional to the height of the tree.
And if we can guarantee that the tree is sufficiently bushy, the height is proportional to $\log n$, where $n$ is the number of elements in the tree.

Later, we'll implement a heap, and we'll see where these performance results come from.
But we'll start by using an implementation in the standard Python library, `heapq`.

`heapq` provides the following functions:

* `heapify`, which takes a list of elements and creates a heap.

* `heappush`, which adds a new element to a heap.

* `heappop`, which removes and returns the element in the heap with highest priority.

Rather than use these functions directly, we will wrap them in a class called `Heap` that provides a more object-oriented interface.

In [30]:
import heapq

class Heap(list):
    """This is a wrapper class for the heap functions provided
    by the heapq module.
    """
    __slots__ = ()
    
    def __init__(self, t=[]):
        """Create a new heap."""
        self.extend(t)
        self.heapify()

    push = lambda x,y: heapq.heappush(x, y)
    pop = lambda x: heapq.heappop(x)
    pushpop = lambda x,y: heapq.heappushpop(x, y)
    replace = lambda x: heapq.heapreplace(x)
    heapify = lambda x: heapq.heapify(x)

    def generator(self):
        """Iterate through the items destructively"""
        while self:
            yield self.popmin()

    def reduce(self, pos, newitem):
        "Replace self[pos] with a lower value item and then reheapify"
        while pos > 0:
            parentpos = (pos - 1) >> 1
            parent = self[parentpos]
            if parent <= newitem:
                break
            self[pos] = parent
            pos = parentpos
        self[pos] = newitem

    def is_heap(self):
        "Return True if the heap has the heap property; False otherwise"
        n = len(self)
        # The largest index there's any point to looking at
        # is the largest with a child index in-range, so must have 2*i + 1 < n,
        # or i < (n-1)/2.  If n is even = 2*j, this is (2*j-1)/2 = j-1/2 so
        # j-1 is the largest, which is n//2 - 1.  If n is odd = 2*j+1, this is
        # (2*j+1-1)/2 = j so j-1 is the largest, and that's again n//2-1.
        try:
            for i in range(n//2):
                if self[i] > self[2*i+1]: return False
                if self[i] > self[2*i+2]: return False
        except IndexError:
            pass
        return True
        

def heapsort(iterable):
    return [x for x in Heap(iterable).generator()]

In [31]:
from random import randint, shuffle

# generate a random test case
n = 15
data = [randint(1,n) for i in range(n)]
shuffle(data)
data

[11, 14, 15, 7, 8, 2, 3, 12, 14, 15, 13, 14, 10, 7, 8]

In [32]:
# test the constructor
heap = Heap(data)
print(heap)

[2, 7, 3, 12, 8, 10, 7, 14, 14, 15, 13, 14, 15, 11, 8]


In [33]:
assert heap.is_heap()

In [34]:
# test popmin
t = []
while heap:
    t.append(heap.popmin())

assert t == sorted(data)

In [35]:
# test 2
shuffle(data)

# test push
heap = Heap()
for item in data:
    heap.push(item)
    
assert heap.is_heap()
print(heap)

[2, 3, 7, 10, 7, 8, 8, 13, 11, 15, 14, 12, 15, 14, 14]


In [36]:
# test 3
shuffle(data)
heap = Heap(data)

# test reduce
for i in range(5):
    pos = randint(0,n-1)
    decr = randint(1,10)
    item = heap[pos] - decr
    heap.reduce(pos, item)
    
assert heap.is_heap()

In [40]:
t = heapsort(data)
assert t == sorted(data)

## Huffman Code

In [41]:
text = 'this is an example of a huffman tree'

In [ ]:
from collections import Counter

c = Counter(text)
c

In [54]:
from collections import namedtuple

Node = namedtuple('Node', ['count', 'letter', 'left', 'right'])

In [57]:
Node(0, 'a', None, None)

Node(count=0, letter='a', left=None, right=None)

In [102]:
nodes = [Node(count, letter, None, None) for (letter, count) in c.most_common()]
nodes

[Node(count=7, letter=' ', left=None, right=None),
 Node(count=4, letter='a', left=None, right=None),
 Node(count=4, letter='e', left=None, right=None),
 Node(count=3, letter='f', left=None, right=None),
 Node(count=2, letter='t', left=None, right=None),
 Node(count=2, letter='h', left=None, right=None),
 Node(count=2, letter='i', left=None, right=None),
 Node(count=2, letter='s', left=None, right=None),
 Node(count=2, letter='n', left=None, right=None),
 Node(count=2, letter='m', left=None, right=None),
 Node(count=1, letter='x', left=None, right=None),
 Node(count=1, letter='p', left=None, right=None),
 Node(count=1, letter='l', left=None, right=None),
 Node(count=1, letter='o', left=None, right=None),
 Node(count=1, letter='u', left=None, right=None),
 Node(count=1, letter='r', left=None, right=None)]

In [107]:
heap = Heap(nodes)
heap

[Node(count=1, letter='l', left=None, right=None),
 Node(count=1, letter='r', left=None, right=None),
 Node(count=1, letter='o', left=None, right=None),
 Node(count=2, letter='n', left=None, right=None),
 Node(count=1, letter='x', left=None, right=None),
 Node(count=1, letter='p', left=None, right=None),
 Node(count=1, letter='u', left=None, right=None),
 Node(count=2, letter='s', left=None, right=None),
 Node(count=4, letter='a', left=None, right=None),
 Node(count=2, letter='m', left=None, right=None),
 Node(count=2, letter='t', left=None, right=None),
 Node(count=4, letter='e', left=None, right=None),
 Node(count=2, letter='h', left=None, right=None),
 Node(count=2, letter='i', left=None, right=None),
 Node(count=7, letter=' ', left=None, right=None),
 Node(count=3, letter='f', left=None, right=None)]

In [108]:
def make_tree(heap):
    while len(heap) > 1:
        left = heap.popmin()
        right = heap.popmin()
        count = left.count + right.count
        node = Node(count, '\0', left, right)
        heap.push(node)
        
    return heap.popmin()

In [109]:
tree = make_tree(heap)

In [141]:
def make_table(tree, path, table):
    if tree is None:
        return
    
    if tree.letter != '\0':
        table[tree.letter] = path
        return
    
    make_table(tree.left, path+'0', table)
    make_table(tree.right, path+'1', table)

In [142]:
table = {}
make_table(tree, '', table)

In [143]:
table

{'l': '00000',
 'o': '00001',
 'p': '00010',
 'r': '00011',
 'u': '00100',
 'x': '00101',
 'h': '0011',
 'i': '0100',
 'm': '0101',
 'n': '0110',
 's': '0111',
 'a': '100',
 'e': '101',
 't': '1100',
 'f': '1101',
 ' ': '111'}

In [144]:
def encode(s, table):
    t = [table[letter] for letter in s]
    return ''.join(t)

In [159]:
code = encode('this is spinal tap',table)
code

'1100001101000111111010001111110111000100100011010000000111110010000010'

In [160]:
def decode(s, tree):
    result = []
    node = tree
    for c in s:
        if c == '0':
            node = node.left
        else:
            node = node.right
            
        if node.letter != '\0':
            result.append(node.letter)
            node = tree
            
    return ''.join(result)

In [161]:
decode(code, tree)

'this is spinal tap'